## Package Imports

In [88]:
import pandas as pd
import numpy as np
import re
import json
import boto3
import s3fs
from azure.storage.blob import BlobServiceClient, generate_account_sas, ResourceTypes, AccountSasPermissions
import tables
from datetime import datetime, timedelta


## Read dataset from public cloud storage

In [89]:
# def read_prefix_to_df(prefix):
#     s3 = boto3.resource('s3')
#     bucket = s3.Bucket('test-bucket')
#     prefix_objs = bucket.objects.filter(Prefix=prefix)
#     prefix_df = []
#     for obj in prefix_objs:
#         key = obj.key
#         body = obj.get()['Body'].read()
#         df = pd.DataFrame(body)
#         prefix_df.append(df)
#     return pd.concat(prefix_df)

In [90]:
data_location = input("From which location we are reading data?").lower()
if data_location in ["aws","s3","amazon"]:
    AWS_S3_BUCKET = input("Your file location please")
    file_name = input("Your full file name please (with extension)")
    if "csv" in file_name.split("."):
        file_name = file_name.split(".")
        df = pd.read_csv(f"s3://{AWS_S3_BUCKET}/{file_name[0]}.csv")
    elif "parquet" or "pqt" in file_name.split("."):
        file_name = file_name.split(".")
        df = pd.read_parquet(f"s3://{AWS_S3_BUCKET}/de/{file_name[0]}.parquet")
    else:
        print("Unavailable file format")
# Not confirmed yet to be developed by reading mechanisim
elif data_location in ["azure","blob"]:
    ACCOUNT_NAME = input("Your Azure account name please")
    STORAGE_ACCOUNT_NAME = input("Your Azure account storage name please")
    ACCOUNT_KEY = input("Your Azure account key please")
    file_name = input("Your full file name please (with extension)")
    blob_name = input("Your blob name please")
    container_name = input("Your container name please")
    # Creating temp sas token
    sas_token = generate_account_sas(
    account_name="{STORAGE_ACCOUNT_NAME}",
    account_key="{ACCOUNT_KEY}",
    resource_types=ResourceTypes(service=True),
    permission=AccountSasPermissions(read=True),
    expiry=datetime.utcnow() + timedelta(hours=1)
    )
    # Creating service client var 
    blob_service_client = BlobServiceClient(account_url=f"https://{ACCOUNT_NAME}.blob.core.windows.net"
                                            , credential=sas_token)
    blob_service_client.get_blob_to_path(container_name,blob_name,file_name)
    if "csv" in file_name.split("."):
        file_name = file_name.split(".")
        df = pd.read_csv(f"{file_name}")
    elif "parquet" or "pqt" in file_name.split("."):
        file_name = file_name.split(".")
        df = pd.read_parquet(f"{file_name}")
    else:
        print("Unavailable file format")
else:
    print("no connector available from given file source location")



## General Details over Dataset

### Head rows of Dataset

In [91]:
head_df = df.head(3)
display(head_df)

,event_id,event_time,user_id,event.payload
0,b9de71c5c3cc4cd7a97e50b832106e5a,2017-06-26 11:23:39,178481.0,"{""event_name"":""view_item"",""platform"":""android""..."
1,23267713c9ea44419331731f50b6a8db,2017-06-27 10:46:39,178481.0,"{""event_name"":""view_item"",""platform"":""android""..."
2,1b7822fa7b854e01970218ae8f721fe0,2017-06-27 11:15:39,178481.0,"{""event_name"":""view_item"",""platform"":""android""..."


### Column data types

In [92]:
col_types = df.dtypes
df_col_types = pd.DataFrame([col_types.values],columns=[df.columns])
display(df_col_types)

,event_id,event_time,user_id,event.payload
0,object,object,float64,object


### Size of Dataset

In [93]:
shape_df = df.shape
df_shape = pd.DataFrame([shape_df],columns = ["rows","cols"])
display(df_shape)

,rows,cols
0,853640,4


### Non numerical columns

In [94]:
describe_non_numerical = df.describe(exclude=[np.number])
display(describe_non_numerical)

,event_id,event_time,event.payload
count,853640,853640,853640
unique,428364,266020,11891
top,b9de71c5c3cc4cd7a97e50b832106e5a,2013-05-25 15:22:54,"{""event_name"":""test_assignment"",""platform"":""se..."
freq,2,2256,82693


### All columns general stats

In [95]:
describe_all_cols = df.describe(include='all')
display(describe_all_cols)

,event_id,event_time,user_id,event.payload
count,853640,853640,853640.000000,853640
unique,428364,266020,NaN,11891
top,b9de71c5c3cc4cd7a97e50b832106e5a,2013-05-25 15:22:54,NaN,"{""event_name"":""test_assignment"",""platform"":""se..."
freq,2,2256,NaN,82693
mean,NaN,NaN,148577.068862,NaN
std,NaN,NaN,81385.659168,NaN
min,NaN,NaN,4.000000,NaN
25%,NaN,NaN,78613.000000,NaN
50%,NaN,NaN,153956.000000,NaN
75%,NaN,NaN,217284.000000,NaN


### Detect candidate key columns (Unique columns)

In [96]:
unique_val_holding_cols = [i for i in df.columns if len(df[i]) == len(pd.unique(df[i]))]
df_unique = pd.DataFrame(unique_val_holding_cols,columns = ["unique_val_holding_cols"])
display(df_unique)

,unique_val_holding_cols


### Null value counts per column

In [97]:
null_val_holding_cols = [df[i].isna().sum() for i in df.columns]
df_null = pd.DataFrame([null_val_holding_cols],columns = df.columns)
display(df_null)

,event_id,event_time,user_id,event.payload
0,0,0,0,0


### Detect JSON Column by col values

In [98]:
# Detect JSON Column by col name
# col_with_dot_char = [df[i].name for i in df.columns if "." in df[i].name]
# print(col_with_dot_char)

col_with_json_val = [df[i].name for i in df.columns if "{" in str(df[i].iloc[0])]
df_json_cols = pd.DataFrame(col_with_json_val,columns = ["col_with_json_val"])
display(df_json_cols)

,col_with_json_val
0,event.payload


### Detect XML Column by col values

In [99]:
col_with_xml_val = [df[i].name for i in df.columns if "<" in str(df[i].iloc[0])]
df_xml_cols = pd.DataFrame(col_with_xml_val,columns = ["col_with_xml_val"])
display(df_xml_cols)

,col_with_xml_val


### Detect Date-Time Columns

In [100]:
#Regex pattern detect date-time columns
pattern_dt = re.compile(r"[0-9]{4}.[0-9]{2}.[0-9]{2}\s[0-9]{2}.[0-9]{2}.[0-9]{2}(\.[0-9]{1,3})?", re.IGNORECASE)
#Loop through columns detect date-time columns
col_date_time = [df[i].name for i in df.columns if pattern_dt.match(str(df[i].iloc[0]))]
df_date_time_cols = pd.DataFrame(col_date_time,columns = ["col_date_time"])
display(df_date_time_cols)

,col_date_time
0,event_time


### Detect Date Columns

In [101]:
#Regex pattern for date columns
pattern_d = re.compile(r"[0-9]{4}.[0-9]{2}.[0-9]{2}.*", re.IGNORECASE)
#Loop through columns detect date columns
col_date = [df[i].name for i in df.columns if pattern_d.match(str(df[i].iloc[0]))]
df_date_cols = pd.DataFrame(col_date,columns = ["col_date"])
display(df_date_cols)

,col_date_time
0,event_time


### Min - Max Date/Time && Difference between Min/Max Date/Time

In [ ]:
if size(vars()[df_date_cols]) > 0 or size(vars()[df_date_time_cols]) > 0:
    min_max_date = [df[i].min() for i in df.columns if pattern_d.match(str(df[i].iloc[0]))]
    min_max_date.extend([df[i].max() for i in df.columns if pattern_d.match(str(df[i].iloc[0]))])
df_date_min_max = pd.DataFrame([min_max_date],columns = ["min_date","max_date"])
df_date_min_max = df_date_min_max.assign(diff_dates=df_date_min_max['max_date']-df_date_min_max['min_date'])
display(df_date_min_max)
